In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim,  num_heads,ff_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim,
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.add = layers.Add()  # instead of `+` to preserve mask
        self.supports_masking = True

    def call(self, inputs, encoder_outputs):
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, use_causal_mask=True
        )
        out_1 = self.layernorm_1(self.add([inputs, attention_output_1]))

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
        )
        out_2 = self.layernorm_2(self.add([out_1, attention_output_2]))

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(self.add([out_2, proj_output]))

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config
maxlen=30
embed_dim = 128  # Embedding size for each token
num_heads = 8  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer
block_num=3 # block_num=6 in the paper
vocab_size=5000
num_bits=500
B=2
def SC_encoder(inputs,flat_flag=True):
    embedding_layer = PositionalEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
   
    for i in range(block_num):
        x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    if flat_flag:
        x=layers.Flatten()(x)
    x=layers.Dense(num_bits//B,activation='sigmoid')(x)
    return x
def SC_decoder(decoder_inputs,encoded_seq_inputs,flat_flag=True): 
    y=tf.reshape(encoded_seq_inputs,[-1,num_bits//B])
    y=layers.Dense(embed_dim*maxlen)(y)
    y=layers.LayerNormalization(epsilon=1e-6)(y)
    y=layers.Reshape((maxlen, embed_dim))(y)
    x = PositionalEmbedding(maxlen, vocab_size, embed_dim)(decoder_inputs)
    for i in range(block_num):
        x = TransformerDecoder(embed_dim, num_heads, ff_dim)(x, y)
        x = layers.Dropout(0.5)(x)
    decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
    return decoder_outputs
encoder_inputs = keras.Input(shape=(maxlen), dtype="int64")
encoder_outputs = SC_encoder(encoder_inputs)
encoder0 = keras.Model(encoder_inputs, encoder_outputs)
encoder0.summary()
decoder_inputs = keras.Input(shape=(maxlen), dtype="int64")
encoded_seq_inputs = keras.Input(shape=( num_bits//B))
decoder_outputs = SC_decoder(decoder_inputs, encoded_seq_inputs)
decoder0 = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)
decoder0.summary() 
decoder_outputs = decoder0([decoder_inputs, encoder_outputs])

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_35 (InputLayer)       [(None, 30)]              0         
                                                                 
 positional_embedding_14 (P  (None, 30, 128)           643840    
 ositionalEmbedding)                                             
                                                                 
 transformer_block_33 (Tran  (None, 30, 128)           561024    
 sformerBlock)                                                   
                                                                 
 transformer_block_34 (Tran  (None, 30, 128)           561024    
 sformerBlock)                                                   
                                                                 
 transformer_block_35 (Tran  (None, 30, 128)           561024    
 sformerBlock)                                            

In [16]:
import pickle
from europarl import TokenizerWrap
with open('tokenizer_5000_1.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
encoder_input_data=tokenizer.tokens_padded
print(print(encoder_input_data.shape))
encoder_inputs = keras.Input(shape=(None,), dtype="int64")
encoder_outputs=encoder0(encoder_inputs)
decoder_inputs = tf.ones_like(encoder_inputs, dtype="int64")
decoder_outputs=decoder0([decoder_inputs,encoder_outputs])
SC_en_de = keras.Model(
    encoder_inputs, decoder_outputs, name="SC_en_de"
)
SC_en_de.summary()

(667875, 30)
None
Model: "SC_en_de"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_38 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 tf.ones_like_1 (TFOpLambda  (None, None)                 0         ['input_38[0][0]']            
 )                                                                                                
                                                                                                  
 model_14 (Functional)       (None, 250)                  3287162   ['input_38[0][0]']            
                                                                                                  
 model_15 (Functional)       (None, 30, 5000)             5526664   ['tf.

In [ ]:
import os
X = tokenizer.tokens_padded[0:500000]
Y = tokenizer.tokens_padded[500000:]
filepath ='SC/'
if not os.path.exists(filepath):
                os.mkdir(filepath[:-1])
if not os.path.exists(filepath+'/his'):
                os.mkdir(filepath)
class logging(keras.callbacks.Callback):
    def __init__(self, model, path='SC.txt'):
        super(logging, self).__init__()

        self.epochs_since_last_save = 0
        self.model = model
        self.path = path

    def on_epoch_end(self, epoch, logs={}):
        if (epoch == 0):
            with open('SC/his/' + self.path, 'w') as f:
                # f.write(str(np.shape(HH))+'\n')
                f.write('Begin~' + '\n')
        with open('SC/his/' + self.path, 'a') as f:
            # f.write(str(np.shape(HH))+'\n')
            f.write(str(logs.get('loss'))+ '\t'+str(logs.get('val_accuracy')) + '\n')
       # print('SNR:',random_SNR(10))
        if (epoch%10==0):
            encoder0.save_weights(filepath+'SC_en_'+str(epoch)+'.h5')
            decoder0.save_weights(filepath+'SC_de_'+str(epoch)+'.h5')
epochs = 100  # SC_en+SC_de step1
SC_en_de.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
log=logging(SC_en_de)
SC_en_de.fit(X,X,batch_size=128, epochs=epochs, validation_data=(Y,Y),callbacks=[log])

Epoch 1/100
3907/3907 [==============================] - 335s 80ms/step - loss: 3.4969 - accuracy: 0.5038 - val_loss: 2.9381 - val_accuracy: 0.5432
Epoch 2/100
3907/3907 [==============================] - 288s 74ms/step - loss: 2.7170 - accuracy: 0.5693 - val_loss: 2.2220 - val_accuracy: 0.6322
Epoch 3/100
3907/3907 [==============================] - 288s 74ms/step - loss: 2.1041 - accuracy: 0.6461 - val_loss: 1.6970 - val_accuracy: 0.7064
Epoch 4/100
3907/3907 [==============================] - 287s 73ms/step - loss: 1.6442 - accuracy: 0.7108 - val_loss: 1.3022 - val_accuracy: 0.7699
Epoch 5/100
3907/3907 [==============================] - 285s 73ms/step - loss: 1.3008 - accuracy: 0.7677 - val_loss: 1.0262 - val_accuracy: 0.8203
Epoch 6/100
3907/3907 [==============================] - 286s 73ms/step - loss: 1.0340 - accuracy: 0.8126 - val_loss: 0.7676 - val_accuracy: 0.8635
Epoch 7/100
3907/3907 [==============================] - 287s 74ms/step - loss: 0.8085 - accuracy: 0.8523 - val_

3907/3907 [==============================] - 285s 73ms/step - loss: 0.0629 - accuracy: 0.9874 - val_loss: 0.0524 - val_accuracy: 0.9905
Epoch 57/100
3907/3907 [==============================] - 286s 73ms/step - loss: 0.0600 - accuracy: 0.9877 - val_loss: 0.0489 - val_accuracy: 0.9911
Epoch 58/100
3907/3907 [==============================] - 286s 73ms/step - loss: 0.0710 - accuracy: 0.9860 - val_loss: 0.0774 - val_accuracy: 0.9860
Epoch 59/100
3907/3907 [==============================] - ETA: 0s - loss: 0.0716 - accuracy: 0.9857

In [163]:
# encoder0.save_weights(filepath+'SC_en0.h5')
# decoder0.save_weights(filepath+'SC_de0.h5')  
encoder0.load_weights(filepath+'SC_en0.h5')
decoder0.load_weights(filepath+'SC_de0.h5') 
encoder0.trainable=False
decoder0.trainable=False

In [162]:
SC_en_de.evaluate(Y,Y)

 100/5247 [..............................] - ETA: 1:12 - loss: 0.0368 - accuracy: 0.9915

KeyboardInterrupt: 

In [138]:
import numpy as np
B=2
N=16
d_model=128
NUM_FEEDBACK_BITS=500
def Num2Bit(Num, B):
    Num_ = Num.numpy()
    bit = (np.unpackbits(np.array(Num_, np.uint8), axis=1).reshape(-1, Num_.shape[1], 8)[:, :, (8-B):]).reshape(-1,
                                                                                                            Num_.shape[
                                                                                                                1] * B)
    bit.astype(np.float32)
    return tf.convert_to_tensor(bit, dtype=tf.float32)
# Bit to Number Function Defining
def Bit2Num(Bit, B):
    Bit_ = Bit.numpy()
    Bit_.astype(np.float32)
    Bit_ = np.reshape(Bit_, [-1, int(Bit_.shape[1] / B), B])
    num = np.zeros(shape=np.shape(Bit_[:, :, 1]))
    for i in range(B):
        num = num + Bit_[:, :, i] * 2 ** (B - 1 - i)
    return tf.cast(num, dtype=tf.float32)
#=======================================================================================================================
#=======================================================================================================================
# Quantization and Dequantization Layers Defining
@tf.custom_gradient
def QuantizationOp(x, B):
    step = tf.cast((2 ** B), dtype=tf.float32)
    result = tf.cast((tf.round(x * step - 0.5)), dtype=tf.float32)
    dim = result.shape[1]
    result = tf.py_function(func=Num2Bit, inp=[result, B], Tout=tf.float32)
    result = tf.reshape(result, [-1, NUM_FEEDBACK_BITS])
    def custom_grad(dy):
        grad = dy
        return (grad, grad)
    return result, custom_grad
class QuantizationLayer(tf.keras.layers.Layer):
    def __init__(self, B,**kwargs):
        self.B = B
        super(QuantizationLayer, self).__init__()
    def call(self, x):
        return QuantizationOp(x, self.B)
    def get_config(self):
        # Implement get_config to enable serialization. This is optional.
        base_config = super(QuantizationLayer, self).get_config()
        base_config['B'] = self.B
        return base_config
@tf.custom_gradient
def DequantizationOp(x, B):
    dim = x.shape[1]
    x = tf.py_function(func=Bit2Num, inp=[x, B], Tout=tf.float32)
    x = tf.reshape(x, (-1, NUM_FEEDBACK_BITS//B))
    step = tf.cast((2 ** B), dtype=tf.float32)
    result = tf.cast((x + 0.5) / step, dtype=tf.float32)
    def custom_grad(dy):
        grad = dy * 1
        return (grad, grad)
    return result, custom_grad
class DeuantizationLayer(tf.keras.layers.Layer):
    def __init__(self, B,**kwargs):
        self.B = B
        super(DeuantizationLayer, self).__init__()
    def call(self, x):
        return DequantizationOp(x, self.B)
    def get_config(self):
        base_config = super(DeuantizationLayer, self).get_config()
        base_config['B'] = self.B
        return base_config
@tf.custom_gradient
def BSCOp(bits, rate):
    errorbits = tf.keras.backend.random_binomial(shape=tf.shape(bits), p=rate)
    result=tf.reshape(tf.math.floormod(bits+errorbits,2),tf.shape(bits))
    def custom_grad(dy):
        grad = dy * 1
        return (grad, grad)
    return result, custom_grad
class BSC(tf.keras.layers.Layer):
    def __init__(self, rate, **kwargs): #**kwargs```TypeError: __init__() got an unexpected keyword argument 'name'```
        self.rate = rate
        super(BSC, self).__init__()
    def call(self, bits):
        
        return BSCOp(bits,self.rate)


In [119]:
test_input=keras.Input(shape=(30,))
a=BSC(0.5)(test_input)
test_model=keras.Model(test_input,a)

In [120]:
test_model.predict(np.ones([1,30]))

1/1 [==============================] - 0s 65ms/step


array([[1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
        0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0.]],
      dtype=float32)

In [135]:
X_codeword=encoder0.predict(X)
Y_codeword=encoder0.predict(Y)
print(X_codeword.shape)

5247/5247 [==============================] - 24s 5ms/step
(500000, 250)


In [172]:
encoder_inputs = keras.Input(shape=(30,), dtype="int64")
SC_en_codeword = encoder0(encoder_inputs)
Q_input=layers.Dense(NUM_FEEDBACK_BITS//B,activation='sigmoid')(SC_en_codeword)
bits=QuantizationLayer(B)(Q_input)#transmission bits
Qmodel=keras.Model(SC_en_codeword,bits)
Qmodel.summary()
rx_bits=BSC(0.0)(bits)
Q_output=DeuantizationLayer(B)(rx_bits)
SC_de_codeword=layers.Dense(NUM_FEEDBACK_BITS//B,activation='sigmoid')(Q_output)
deQmodel=keras.Model(rx_bits,SC_de_codeword)
deQmodel.summary()
decoder_inputs = tf.ones_like(encoder_inputs, dtype="int64")
decoder_outputs=decoder0([decoder_inputs,SC_de_codeword])
Q_deQ = keras.Model(
    SC_en_codeword, SC_de_codeword, name="Q_deQ"
)
Q_deQ.summary()
SC_en_Q_deQ_SC_de = keras.Model(
    encoder_inputs, decoder_outputs, name="SC_en_Q_deQ_SC_de"
)
SC_en_Q_deQ_SC_de.summary()

Model: "model_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_99 (InputLayer)       [(None, 250)]             0         
                                                                 
 dense_126 (Dense)           (None, 250)               62750     
                                                                 
 quantization_layer_24 (Qua  (None, 500)               0         
 ntizationLayer)                                                 
                                                                 
Total params: 62750 (245.12 KB)
Trainable params: 62750 (245.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "model_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_100 (InputLayer)      [(None, 500)]             0         
  

In [ ]:
# X_codeword=encoder0.predict(X)
# Y_codeword=encoder0.predict(Y)
# print(X_codeword.shape)
SC_en_Q_deQ_SC_de.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
filepath ='SC/'
if not os.path.exists(filepath):
                os.mkdir(filepath[:-1])
if not os.path.exists(filepath+'/his'):
                os.mkdir(filepath)
class logging(keras.callbacks.Callback):
    def __init__(self, model, path='Q_deQ.txt'):
        super(logging, self).__init__()

        self.epochs_since_last_save = 0
        self.model = model
        self.path = path

    def on_epoch_end(self, epoch, logs={}):
        if (epoch == 0):
            with open('SC/his/' + self.path, 'w') as f:
                # f.write(str(np.shape(HH))+'\n')
                f.write('Begin~' + '\n')
        with open('SC/his/' + self.path, 'a') as f:
            # f.write(str(np.shape(HH))+'\n')
            f.write(str(logs.get('loss'))+ '\t'+str(logs.get('val_accuracy')) + '\n')
        SC_en_Q_deQ_SC_de.evaluate(Y[:100],Y[:100])
        if (epoch%10==0):
            
            Qmodel.save_weights(filepath+'Q_model_'+str(epoch)+'.h5')
            deQmodel.save_weights(filepath+'deQ_model_'+str(epoch)+'.h5')
epochs = 100  # SC_en+SC_de step1
Q_deQ.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss="mse", metrics=["mse"]
)
log=logging(Q_deQ)
Q_deQ.fit(X_codeword,X_codeword,batch_size=128, epochs=epochs,callbacks=[log])

Epoch 1/100
1950/3907 [=============>................] - ETA: 14s - loss: 0.0156 - mse: 0.0156

In [178]:
# encoder0.trainable=False
# decoder0.trainable=False
# filepath ='SC/'
# if not os.path.exists(filepath):
#                 os.mkdir(filepath[:-1])
# if not os.path.exists(filepath+'/his'):
#                 os.mkdir(filepath)
# class logging(keras.callbacks.Callback):
#     def __init__(self, model, path='Q_SC.txt'):
#         super(logging, self).__init__()

#         self.epochs_since_last_save = 0
#         self.model = model
#         self.path = path

#     def on_epoch_end(self, epoch, logs={}):
#         if (epoch == 0):
#             with open('SC/his/' + self.path, 'w') as f:
#                 # f.write(str(np.shape(HH))+'\n')
#                 f.write('Begin~' + '\n')
#         with open('SC/his/' + self.path, 'a') as f:
#             # f.write(str(np.shape(HH))+'\n')
#             f.write(str(logs.get('loss'))+ '\t'+str(logs.get('val_accuracy')) + '\n')
#        # print('SNR:',random_SNR(10))
#         if (epoch%10==0):
#             encoder0.save_weights(filepath+'F_SC_en_'+str(epoch)+'.h5')
#             decoder0.save_weights(filepath+'F_SC_de_'+str(epoch)+'.h5')
#             Qmodel.save_weights(filepath+'F_Q_model_'+str(epoch)+'.h5')
#             deQmodel.save_weights(filepath+'F_de_Q_model_'+str(epoch)+'.h5')
# epochs = 100  # SC_en+SC_de step1
# SC_en_Q_deQ_SC_de.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"]
# )
# log=logging(SC_en_Q_deQ_SC_de)

# encoder0.trainable=True
# decoder0.trainable=True
# Qmodel.trainable=True
# deQmodel.trainable=True
# SC_en_Q_deQ_SC_de.summary()
#  SC_en_Q_deQ_SC_de.fit(X,X,batch_size=128, epochs=10, validation_data=(Y,Y),callbacks=[log])

In [177]:
SC_en_Q_deQ_SC_de.evaluate(Y[:100],Y[:100])
result=np.argmax(SC_en_Q_deQ_SC_de.predict(Y[:100]),-1)
print(result.shape)
tokenizer.tokens_to_string(result[0])

4/4 [==============================] - 0s 17ms/step
(100, 30)


'yet we are supposed to allow these countries into schengen'